In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = 'gpt-4o')

In [5]:
from langchain_community.document_loaders import JSONLoader
loader = JSONLoader(file_path='Test.json', jq_schema='.content' )